WHAT IS THE BEST LOCATION IN MANHATTAN TO OPEN A COFFEE STALL?

INTRODUCTION

The problem or question to be asked is "what is the best neighbourhood of Manhattan to open a coffee stall?"

Good locations are those with a high foot traffic but with the lowest competition.

A good area would have a high foot traffic, but a low number of coffee shops.

Audience of the results will be entrepreneurs looking to set up a coffee stall in Manhattan.

They care about this question/problem because Manhattan is already a competitve place for coffee shops, but opening shop in a neighbourhood with high foot traffic and low competition will be the area most likely to return a profit.

In [1]:
!conda install lxml --yes

import pandas as pd

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2020.6.2~ --> pkgs/main::ca-certificates-2020.6.24-0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2020.6.20-py36h9~ --> pkgs/main::certifi-2020.6.20-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_1 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ con

DATA

Three datasets willl be used:

1) List of Manhattan neighbourhoods from wikipedia

https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods#:~:text=The%20following%20approximate%20definitions%20are,34th%20Street%20and%2059th%20Street.

bringing in the following string data

In [19]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_Manhattan_neighborhoods', header=0)
pd.set_option('display.max_rows', 200)
df_upper = data[0]
df_midtown = data[1]
df_between = data[2]
df_downtown = data[3]
frames = [df_upper, df_midtown, df_between, df_downtown]
df_manhattan = pd.concat(frames)

Examples of Manhattan neighbourhoods

In [20]:
df_manhattan.head()

,Name of the neighborhood,Limits south to north and east to west
0,Upper Manhattan,Above 96th Street
1,Marble Hill,Physically located on the mainland
2,Inwood,Above Dyckman Street
3,Fort George (part of Washington Heights),East of Broadway between 181st Street and Dyck...
4,Washington Heights,155th Street to Dyckman Street


Description of Manhattan Neighbourhoods

In [22]:
df_manhattan.describe()

,Name of the neighborhood,Limits south to north and east to west
count,85,85
unique,83,85
top,Hudson Yards,40th to 59th Streets; 3rd to 9th Avenues
freq,2,1


2) Pedestrian Traffic data from NYC Open Data

https://data.cityofnewyork.us/api/views/cqsj-cfgu/rows.csv?accessType=DOWNLOAD&bom=true&format=true

The dataset has the following relevant columns:

Borough (e.g. Manhattan) - String
the_geom (lat, lng coordinate) - String
Time series columns (e.g. 1576 for May 2019 at 161 street in the bronx) - Integer

For each year from 2007 to 2019 measurements of pedestrian data is taken bi annually in May and September at morning, midday and afternoon. 

Data for 2019 only will be averaged for each neighbourhood. 

The GeoPy Library will be used to assign an area name to each lat and lng value in the Pedestrian Traffic Data using the Reverse function. If a matching neighbourhood address cannot be assigned, one will be assigned manually.

In [49]:
df_ped = pd.read_csv('https://data.cityofnewyork.us/api/views/cqsj-cfgu/rows.csv?accessType=DOWNLOAD&bom=true&format=true')
df_ped = df_ped[(df_ped.Borough=='Manhattan')]

Examples of pedestrian Traffic data 

In [50]:
df_ped.head()

,Borough,the_geom,OBJECTID,Loc,Street_Nam,From_Stree,To_Street,Index,May07_AM,May07_PM,...,Sept17_PM,Sept17_MD,May18_AM,May18_PM,May18_MD,Sept18_PM,Sept18_MD,May19_AM,May19_PM,May19_MD
34,Manhattan,POINT (-74.01155687409947 40.70463665187371),35,35,Broad Street,Beaver Street,South William Street,Y,3469,3992,...,"8,303","2,036","4,374","6,603","1,756","6,471","2,010","4,100","7,302","1,669"
35,Manhattan,POINT (-74.01286204592034 40.70634164448266),36,36,Broadway,Morris Street,Exchange Place,Y,3660,8390,...,"12,650","8,126","5,221","19,725","6,818","10,726","9,615","5,049","11,765","7,029"
36,Manhattan,POINT (-73.98219706247882 40.77181340301184),37,37,Broadway,West 63rd Street,West 64th Street,Y,1611,6764,...,"9,305","4,663","2,059","6,194","6,037","7,773","5,259","1,696","6,864","4,907"
37,Manhattan,POINT (-74.01009312926121 40.715904559004194),38,38,Chambers Street,West Broadway,Greenwich Street,Y,7081,8512,...,"9,937","3,302","8,323","8,960","3,630","10,456","3,493","3,075","6,598","2,934"
38,Manhattan,POINT (-73.97713579908014 40.7796808276313),39,39,Columbus Avenue,West 75th Street,West 76th Street,N,1071,3037,...,"3,626","2,977","1,524","3,905","3,780","3,794","2,451",-,-,-


Description of pedestrian traffic data

In [51]:
df_ped.describe()

,OBJECTID,Loc,May09_AM,May09_PM,May09_MD,May10_AM,May10_PM,May10_MD,Sept10_AM,Sept10_PM,...,May11_MD,Sept11_AM,Sept11_PM,Sept11_MD,May12_AM,May12_PM,May12_MD,Sept15_AM,Sept15_PM,Sept15_MD
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,...,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000
mean,52.500000,52.500000,3709.666667,9949.138889,4485.750000,4096.194444,11097.277778,4615.277778,4033.916667,10709.888889,...,4989.111111,4318.944444,10446.722222,5140.527778,4120.861111,7514.805556,5268.638889,4492.611111,11652.833333,4686.000000
std,10.535654,10.535654,2664.188829,6881.739540,3273.855022,2911.466757,6968.435994,2991.685064,2498.866673,6727.910071,...,3196.104690,3210.427448,5806.368910,3488.050151,2830.395946,4576.350004,3474.177437,2505.843482,5938.561180,3008.181302
min,35.000000,35.000000,27.000000,192.000000,64.000000,280.000000,453.000000,201.000000,273.000000,313.000000,...,487.000000,233.000000,141.000000,1515.000000,255.000000,118.000000,1175.000000,281.000000,694.000000,733.000000
25%,43.750000,43.750000,1543.750000,5762.250000,2240.500000,1986.500000,6151.750000,2591.750000,2378.500000,6141.500000,...,2700.500000,1993.250000,5882.000000,2435.750000,2101.000000,4280.250000,2704.500000,2349.000000,7205.250000,2191.000000
50%,52.500000,52.500000,3424.000000,8518.000000,3557.500000,3907.500000,9961.000000,3831.000000,3693.500000,9001.000000,...,4216.500000,3759.000000,10063.500000,4401.000000,3753.000000,6798.000000,4333.500000,4088.000000,11341.500000,3625.000000
75%,61.250000,61.250000,4778.750000,11090.750000,5465.000000,5312.000000,13007.250000,5981.750000,4782.750000,13997.750000,...,6006.500000,5812.500000,12380.750000,6709.000000,5608.500000,8990.000000,6343.500000,6306.250000,13843.250000,6524.000000
max,70.000000,70.000000,12690.000000,29526.000000,13971.000000,13421.000000,30544.000000,12727.000000,10010.000000,30103.000000,...,14182.000000,14456.000000,27249.000000,15946.000000,13645.000000,18969.000000,13790.000000,10197.000000,25687.000000,11670.000000


3) Coffee shop venues for each neighbourhood from Foursquare API. 

The venue search query will be used to find the number of coffee shops within each neighbourhood. The call will return a JSON file. The JSON file will return venue data such as name, unique id, category and location.

Location will contain address, country, lat, lng, and distance. All fields are strings, except for lat, lng, and distance. Distance is measured in meters. Some venues have their locations hidden for privacy reasons. 

Category is an array of categories that have been applied to a venue. One of the categories will have a primary field.

JSON data will be returned from the foursquare API called and the number of coffee venues will be added as as column to the df_manhattan dataframe. Data under response, venues, name will be counted.

Initalising foursquare:

In [43]:
CLIENT_ID = 'DMQJLIDJ0EYXZOY2VVA52MB1A5HHX03WA5S0YF54QQDIVCW5' # your Foursquare ID
CLIENT_SECRET = 'MZTDY3U4BC2AGDMNAFBSOH34SHPVYDVKNYND32WBOETW3A1I' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

Example foursquare API call for coffee shops:

In [46]:
LIMIT = 1 # limit of number of venues returned by Foursquare API
NEAR = 'Upper Manhattan'
CATID = '4bf58dd8d48988d1e0931735'

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&near={},&limit={}&categoryId={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
NEAR,
LIMIT,
CATID)
            
# make the GET request
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5f42a7907f58eb2f2fe25e78'},
 'response': {'venues': [{'id': '58d933702f91cb026f478e38',
    'name': 'East One Coffee Roasters',
    'location': {'address': '384 Court St',
     'crossStreet': 'at Carroll St',
     'lat': 40.681128035266816,
     'lng': -73.99652634325895,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.681128035266816,
       'lng': -73.99652634325895}],
     'postalCode': '11231',
     'cc': 'US',
     'neighborhood': 'Carroll Gardens',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['384 Court St (at Carroll St)',
      'Brooklyn, NY 11231',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1e0931735',
      'name': 'Coffee Shop',
      'pluralName': 'Coffee Shops',
      'shortName': 'Coffee Shop',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_',
       'suffix': '.png'},
      'primary': True}],
    'delivery